In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_21384: Some(http://gw02.itversity.com:4056)

In [2]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1533622723243_21384


In [3]:
import scala.collection.JavaConversions._

In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getProperty("user.home").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home+"/dataSets/medline_2016"

path = /user/kranthidr/dataSets/medline_2016


/user/kranthidr/dataSets/medline_2016

In [6]:
import com.cloudera.datascience.graph.RunGraph._

In [7]:
import edu.umd.cloud9.collection.XMLInputFormat

import java.nio.charset.StandardCharsets
import java.security.MessageDigest

import org.apache.hadoop.io.{Text, LongWritable}
import org.apache.hadoop.conf.Configuration

import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{Dataset, SparkSession, Row}
import org.apache.spark.sql.functions._

import scala.xml._

In [8]:
    val medlineRaw: Dataset[String] = loadMedline(spark, path)

medlineRaw = [value: string]


[value: string]

In [9]:
    val medline: Dataset[Seq[String]] = medlineRaw.map(majorTopics).cache()

medline = [value: array<string>]


[value: array<string>]

In [10]:
medline.printSchema()

root
 |-- value: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [11]:
medline.count()

240000

In [12]:
medline.show(1, false)

+---------------------------------------------------------------------------+
|value                                                                      |
+---------------------------------------------------------------------------+
|[Intellectual Disability, Maternal-Fetal Exchange, Pregnancy Complications]|
+---------------------------------------------------------------------------+
only showing top 1 row



In [13]:
    val topics = medline.flatMap(mesh => mesh).toDF("topic")

topics = [topic: string]


[topic: string]

In [14]:
    topics.createOrReplaceTempView("topics")

In [15]:
    val topicDist = spark.sql("SELECT topic, COUNT(*) cnt FROM topics GROUP BY topic ORDER BY cnt DESC")

topicDist = [topic: string, cnt: bigint]


[topic: string, cnt: bigint]

In [16]:
    topicDist.show()

+--------------------+----+
|               topic| cnt|
+--------------------+----+
|            Research|1649|
|             Disease|1349|
|           Neoplasms|1123|
|        Tuberculosis|1066|
|       Public Policy| 816|
|       Jurisprudence| 796|
|          Demography| 763|
| Population Dynamics| 753|
|           Economics| 690|
|            Medicine| 682|
|Socioeconomic Fac...| 655|
|            Politics| 631|
|               Blood| 631|
|Emigration and Im...| 601|
|       Social Change| 577|
|          Physicians| 560|
|            Mutation| 542|
|   Abortion, Induced| 503|
|          Anesthesia| 483|
|       Public Health| 479|
+--------------------+----+
only showing top 20 rows



In [17]:
    topicDist.createOrReplaceTempView("topic_dist")

In [18]:
    spark.sql("SELECT cnt, COUNT(*) dist FROM topic_dist GROUP BY cnt ORDER BY dist DESC LIMIT 10").show()

+---+----+
|cnt|dist|
+---+----+
|  1|3106|
|  2|1699|
|  3|1207|
|  4| 902|
|  5| 680|
|  6| 571|
|  7| 490|
|  8| 380|
|  9| 356|
| 10| 296|
+---+----+



In [19]:
    val topicPairs = medline.flatMap(_.sorted.combinations(2)).toDF("pairs")

topicPairs = [pairs: array<string>]


[pairs: array<string>]

In [20]:
    topicPairs.createOrReplaceTempView("topic_pairs")

In [21]:
    val cooccurs = spark.sql("SELECT pairs, COUNT(*) cnt FROM topic_pairs GROUP BY pairs")
    cooccurs.cache()

cooccurs = [pairs: array<string>, cnt: bigint]


[pairs: array<string>, cnt: bigint]

In [22]:
    cooccurs.createOrReplaceTempView("cooccurs")

In [23]:
    println("Number of unique co-occurrence pairs: " + cooccurs.count())

Number of unique co-occurrence pairs: 213745


In [24]:
    spark.sql("SELECT pairs, cnt FROM cooccurs ORDER BY cnt DESC LIMIT 10").show()

+--------------------+---+
|               pairs|cnt|
+--------------------+---+
|[Demography, Popu...|288|
|[Government Regul...|254|
|[Emigration and I...|230|
|[Acquired Immunod...|220|
|[Antibiotics, Ant...|205|
|[Analgesia, Anest...|183|
|[Economics, Popul...|181|
|[Analgesia, Anest...|179|
|[Anesthesia, Anes...|177|
|[Population Dynam...|174|
+--------------------+---+



In [25]:
    val vertices = topics.map { case Row(topic: String) => (hashId(topic), topic) }.toDF("hash", "topic")

vertices = [hash: bigint, topic: string]


[hash: bigint, topic: string]

In [26]:
    val edges = cooccurs.map { case Row(topics: Seq[_], cnt: Long) =>
       val ids = topics.map(_.toString).map(hashId).sorted
       Edge(ids(0), ids(1), cnt)
    }

edges = [srcId: bigint, dstId: bigint ... 1 more field]


[srcId: bigint, dstId: bigint ... 1 more field]

In [27]:
    val vertexRDD = vertices.rdd.map{ case Row(hash: Long, topic: String) => (hash, topic) }

vertexRDD = MapPartitionsRDD[90] at map at <console>:61


MapPartitionsRDD[90] at map at <console>:61

In [28]:
    val topicGraph = Graph(vertexRDD, edges.rdd)
    topicGraph.cache()

topicGraph = org.apache.spark.graphx.impl.GraphImpl@6f0e2189


org.apache.spark.graphx.impl.GraphImpl@6f0e2189

In [29]:
    val connectedComponentGraph = topicGraph.connectedComponents()

connectedComponentGraph = org.apache.spark.graphx.impl.GraphImpl@40468bb4


org.apache.spark.graphx.impl.GraphImpl@40468bb4

In [30]:
    val componentDF = connectedComponentGraph.vertices.toDF("vid", "cid")

componentDF = [vid: bigint, cid: bigint]


[vid: bigint, cid: bigint]

In [31]:
    val componentCounts = componentDF.groupBy("cid").count()

componentCounts = [cid: bigint, count: bigint]


[cid: bigint, count: bigint]

In [32]:
    componentCounts.orderBy(desc("count")).show()

+--------------------+-----+
|                 cid|count|
+--------------------+-----+
|-9218306090261648869|13610|
|-8193948242717911820|    5|
|-2062883918534425492|    4|
|-7016546051037489808|    3|
| 2742772755763603550|    3|
|-7685954109876710390|    3|
| -784187332742198415|    3|
| 1765411469112156596|    3|
|-8679136035911620397|    3|
|-3299226677350014771|    2|
|-5179176761971077378|    2|
|-2317423407077322989|    2|
|-9211944049288765106|    2|
|-5295884525273097033|    2|
|-3191983795676547449|    2|
| 2026738476704047088|    2|
|-1390944942283085161|    2|
| -697775734067750523|    2|
|-5362458719777034637|    2|
|-3467839743215210439|    2|
+--------------------+-----+
only showing top 20 rows



In [33]:
    val topicComponentDF = topicGraph.vertices.innerJoin(
      connectedComponentGraph.vertices) {
      (topicId, name, componentId) => (name, componentId.toLong)
    }.values.toDF("topic", "cid")

topicComponentDF = [topic: string, cid: bigint]


[topic: string, cid: bigint]

In [34]:
    topicComponentDF.where("cid = -2062883918534425492").show()

+--------------------+--------------------+
|               topic|                 cid|
+--------------------+--------------------+
|          Serotyping|-2062883918534425492|
|Campylobacter jejuni|-2062883918534425492|
|Campylobacter Inf...|-2062883918534425492|
|  Campylobacter coli|-2062883918534425492|
+--------------------+--------------------+



In [35]:
    val campy = spark.sql("SELECT * FROM topic_dist WHERE topic LIKE '%ampylobacter%'")
    campy.show()

+--------------------+---+
|               topic|cnt|
+--------------------+---+
|Campylobacter jejuni|  3|
|Campylobacter Inf...|  2|
| Campylobacter fetus|  1|
|  Campylobacter coli|  1|
|       Campylobacter|  1|
+--------------------+---+



campy = [topic: string, cnt: bigint]


[topic: string, cnt: bigint]

In [36]:
    val degrees: VertexRDD[Int] = topicGraph.degrees.cache()

degrees = VertexRDDImpl[277] at RDD at VertexRDD.scala:57


VertexRDDImpl[277] at RDD at VertexRDD.scala:57

In [37]:
    degrees.map(_._2).stats()
    degrees.innerJoin(topicGraph.vertices) {
      (topicId, degree, name) => (name, degree.toInt)
    }.values.toDF("topic", "degree").orderBy(desc("degree")).show()

+-------------------+------+
|              topic|degree|
+-------------------+------+
|           Research|  2596|
|            Disease|  1746|
|          Neoplasms|  1202|
|              Blood|   914|
|       Pharmacology|   882|
|       Tuberculosis|   815|
|         Toxicology|   694|
|       Drug Therapy|   678|
|      Jurisprudence|   661|
|Biomedical Research|   633|
|         Physicians|   625|
|      Public Policy|   601|
|           Medicine|   590|
|         Metabolism|   578|
|      Social Change|   570|
|Wounds and Injuries|   570|
|              Brain|   569|
|          Hospitals|   557|
|              Urine|   551|
|          Economics|   548|
+-------------------+------+
only showing top 20 rows



In [38]:
    val T = medline.count()

T = 240000


240000

In [39]:
    val topicDistRdd = topicDist.map { case Row(topic: String, cnt: Long) => (hashId(topic), cnt) }.rdd

topicDistRdd = MapPartitionsRDD[313] at rdd at <console>:51


MapPartitionsRDD[313] at rdd at <console>:51

In [40]:
    val topicDistGraph = Graph(topicDistRdd, topicGraph.edges)

topicDistGraph = org.apache.spark.graphx.impl.GraphImpl@58580eaa


org.apache.spark.graphx.impl.GraphImpl@58580eaa

In [41]:
    val chiSquaredGraph = topicDistGraph.mapTriplets(triplet =>
      chiSq(triplet.attr, triplet.srcAttr, triplet.dstAttr, T)
    )

chiSquaredGraph = org.apache.spark.graphx.impl.GraphImpl@422d0e99


org.apache.spark.graphx.impl.GraphImpl@422d0e99

In [42]:
    chiSquaredGraph.edges.map(x => x.attr).stats()

(count: 213745, mean: 877.956648, stdev: 5094.935171, max: 198668.408387, min: 0.000000)

In [43]:
    val interesting = chiSquaredGraph.subgraph(triplet => triplet.attr > 19.5)

interesting = org.apache.spark.graphx.impl.GraphImpl@56f7fba8


org.apache.spark.graphx.impl.GraphImpl@56f7fba8

In [44]:
    val interestingComponentGraph = interesting.connectedComponents()

interestingComponentGraph = org.apache.spark.graphx.impl.GraphImpl@b06bc78


org.apache.spark.graphx.impl.GraphImpl@b06bc78

In [45]:
    val icDF = interestingComponentGraph.vertices.toDF("vid", "cid")

icDF = [vid: bigint, cid: bigint]


[vid: bigint, cid: bigint]

In [46]:
    val icCountDF = icDF.groupBy("cid").count()

icCountDF = [cid: bigint, count: bigint]


[cid: bigint, count: bigint]

In [47]:
    icCountDF.count()

878

In [48]:
    icCountDF.orderBy(desc("count")).show()

+--------------------+-----+
|                 cid|count|
+--------------------+-----+
|-9218306090261648869|13610|
|-8193948242717911820|    5|
|-2062883918534425492|    4|
|-7016546051037489808|    3|
| 2742772755763603550|    3|
|-7685954109876710390|    3|
| -784187332742198415|    3|
| 1765411469112156596|    3|
|-8679136035911620397|    3|
|-3299226677350014771|    2|
|-1390944942283085161|    2|
| 2026738476704047088|    2|
|-9211944049288765106|    2|
|-5295884525273097033|    2|
|-3191983795676547449|    2|
|-1046815223728304871|    2|
| -697775734067750523|    2|
|-5179176761971077378|    2|
|-5362458719777034637|    2|
|-3467839743215210439|    2|
+--------------------+-----+
only showing top 20 rows



In [49]:
    val interestingDegrees = interesting.degrees.cache()

interestingDegrees = VertexRDDImpl[514] at RDD at VertexRDD.scala:57


VertexRDDImpl[514] at RDD at VertexRDD.scala:57

In [50]:
    interestingDegrees.map(_._2).stats()

(count: 13721, mean: 20.490489, stdev: 29.864223, max: 863.000000, min: 1.000000)

In [51]:
    interestingDegrees.innerJoin(topicGraph.vertices) {
      (topicId, degree, name) => (name, degree)
    }.toDF("topic", "degree").orderBy(desc("degree")).show()

+--------------------+--------------------+
|               topic|              degree|
+--------------------+--------------------+
| 7245808948765301120|[von Willebrand D...|
| 5099251899234384801|[src-Family Kinas...|
|-1530704564905827748|[src Homology Dom...|
| 2507157745758004630|[rho GTP-Binding ...|
|-7862235140823909271|       [ras-GRF1, 1]|
| 2059861029745035294|   [ras Proteins, 6]|
| 5955929388309042993|[rab GTP-Binding ...|
| 7816430327900570785|    [rRNA Operon, 2]|
| 7239719559500007147|[para-Aminobenzoa...|
| 1284163606049864457|[p38 Mitogen-Acti...|
|-1087266232809183895|[p-Methoxy-N-meth...|
|-8705584836746698588|[p-Dimethylaminoa...|
|-5100017000906699736|[p-Aminohippuric ...|
|-4066692375147750810|[p-Aminoazobenzen...|
|-1046658263400552818|[ortho-Aminobenzo...|
|-4311340282177857587|[omega-Conotoxins...|
| 6603518327358446980|[gamma-Globulins,...|
|-2996202821866347894|[gamma-Aminobutyr...|
|-6978724293355107821|[cis-trans-Isomer...|
|  172595453653541184|[cdc25 Pho

In [52]:
    val avgCC = avgClusteringCoef(interesting)

avgCC = 0.306246256051886


0.306246256051886

In [53]:
    val paths = samplePathLengths(interesting)
    paths.map(_._3).filter(_ > 0).stats()

paths = MapPartitionsRDD[741] at distinct at RunGraph.scala:142


(count: 3170622, mean: 3.607923, stdev: 0.776411, max: 8.000000, min: 1.000000)

In [54]:
    val hist = paths.map(_._3).countByValue()
    hist.toSeq.sorted.foreach(println)

(0,255)
(1,4834)
(2,168905)
(3,1259106)
(4,1394555)
(5,319149)
(6,23106)
(7,940)
(8,27)


hist = Map(0 -> 255, 5 -> 319149, 1 -> 4834, 6 -> 23106, 2 -> 168905, 7 -> 940, 3 -> 1259106, 8 -> 27, 4 -> 1394555)


Map(0 -> 255, 5 -> 319149, 1 -> 4834, 6 -> 23106, 2 -> 168905, 7 -> 940, 3 -> 1259106, 8 -> 27, 4 -> 1394555)